# Libraries

In [146]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re
from docx import Document
import nltk
from itertools import chain
import heapq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import string

In [121]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/korneliusz.krysiak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/korneliusz.krysiak/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Text transformations

## All documents LOOP

In [2]:
#Open all documents and store the path in a list
document_list = []
for i in range(9):
    document = Document('data/raw/Podcast_9{}.docx'.format(i))
    document_list.append(document)

In [3]:
#extract headings
def iter_headings(paragraphs):
    for paragraph in paragraphs:
        if paragraph.style.name.startswith('Heading'):
            yield paragraph

In [4]:
#Big LOOP for all documents
heading_list_big = []
extracted_text_big = []
gamesDict_big = []

for z in document_list:
    
    heading_list = []
    
    #extraction of headings from our documents
    for heading in iter_headings(z.paragraphs):
        heading_list.append(heading.text)

    heading_list = heading_list[1:-1]
    
    text_lines = []

    for i,para in enumerate(z.paragraphs):
        if para.text in heading_list:
            text_lines.append(i)

    #extracting text lines for games only (without intro and outro)       
    gamesText = []

    for k,j in enumerate(text_lines):
        for y, parar in enumerate(z.paragraphs):
            if y>=text_lines[k]:
                gamesText.append(parar.text)

    #creating list mapping where we get the in-text position of heading (number of line)            
    text_lines2 = [text_lines[n]-text_lines[0] for n in range(1,len(text_lines))]
    text_lines2.insert(0,0)
    
    #Removing timestamp from headings
    heading_list2 = []

    for g in heading_list:
        text = g[:-11]
        heading_list2.append(text)

    heading_list_big.append(heading_list2)
    
    #Extracting text with respect to headings: list of dialogs related to specific game
    extracted_text = []

    for d,w in zip(text_lines2,text_lines2[1:]):
        text = gamesText[d+1:w]
        extracted_text.append(text)
        
    gamesDict = dict(zip(heading_list, extracted_text))
    gamesDict_big.append(gamesDict)
    extracted_text_big.append(extracted_text)

In [5]:
#flatten the list of headings
flat_list = []
for sublist in heading_list_big:
    for item in sublist:
        flat_list.append(item)

In [67]:
#tu beda 3 wartosci, bo to jest lista list
# pierwsza to numer pliku, druga to numer planszowki, 3 to kolejne osoby w wywiadzie
#czyli np : 0 0 0 to jest 1 docx 1 planszowka 1 wypowiedź, a 0 0 1 to juz 2 wypodzidz do tej samej planszowki
#print(extracted_text_big[0][0][2])
print(extracted_text_big[0][0])

['Quinns: Let’s talk briefly about Blackout: Hong Kong, a eurogame by Alexander Pfister, which I got very excited about a couple of episodes ago, a game about running an organization in Hong Kong after a power cut and you have to keep the city running, you have to keep medicine and food flowing. You gotta keep that food flowing, Matthew.', 'Matt: And the reasons you were excited about it, you had every right to be excited about, because actually the coolest mechanics in it are very evocative and very cool. The whole thing of having to send people out to scout and recon, but every time you do you’ll lose somebody from your group/gang, depending on what kind of unsavory types you’re hiring: Very interesting! We played it a bunch when we were in Philadelphia. We played it just me and you, you had played with some other people, and to be honest, I found that the more we played it… It’s a very competent puzzle by Mr. Pfister, but it doesn’t have the same kind of thematic flair as Great West

# Text Mining

In [93]:
#Join the reviewers speach about one game as one text
big_joined_list = []
for i in extracted_text_big:
    joined_list = []
    big_joined_list.append(joined_list)
    for j in i:
        joined_list.append(' '.join(j))

In [231]:
#tokenize
tokenized = []
for i in big_joined_list:
    word_tokens = []
    tokenized.append(word_tokens)
    for j in i:
        word_tokens.append(word_tokenize(j))

In [232]:
#unlist
tokenized = list(chain.from_iterable(tokenized))
#set stopwords
stop_words = set(stopwords.words('english'))
#punctiation seting
punctuations = list(string.punctuation)
own_punctuations = ["''","_","“","’","…","[","-","”"]
punctuations.append(own_punctuations)
punctuations = list(chain.from_iterable(punctuations))

In [233]:
#clear tokens from punctuation
cleared_tokens = []
for token in tokenized:
    cleared_tokens.append([i.strip("".join(punctuations)) for i in token if i not in punctuations])

In [219]:
#print punctuation for one game
print(cleared_tokens[1])

['Quinns', 'Let', 's', 'talk', 'about', 'something', 'a', 'bit', 'more', 'positive', 'let', 's', 'talk', 'about', 'Just', 'One', 'Which', 'is', 'so', 'simple', 'I', 'can', 't', 'imagine', 'we', 're', 'gon', 'na', 'be', 'talking', 'about', 'it', 'for', 'a', 'long', 'time', 'but', 'my', 'goodness', 'I', 've', 'been', 'having', 'a', 'lot', 'of', 'fun', 'with', 'this', 'Matt', 'It', 's', 'really', 'quite', 'good', 'Quinns', 'How', 'would', 'you', 'describe', 'Just', 'One', 'Matthew', 'Lees', 'Matt', 'I', 'd', 'describe', 'Just', 'One', 'by', 'saying', 'that', 'it', 's', 'a', 'game', 'where', 'you', 'have', 'You', 'know', 'when', 'you', 'have', 'little', 'placeholders', 'at', 'restaurants', 'or', 'like', 'weddings', '–', 'wedding', 's', 'better', '–', 'where', 'it', 's', 'like', 'a', 'little', 'bit', 'of', 'plastic', 'that', 'effectively', 'has', 'got', 'a', 'name', 'on', 'it', 'except', 'they', 're', 'blank', 'and', 'you', 'have', 'little', 'pens', 'and', 'each', 'time', 'there', 'is', 'on

In [235]:
#removing stop words
filtered_sentences = []
for sentence in cleared_tokens:
    filtered_sentences.append([w for w in sentence if not w in stop_words])

#print filtered sentences
print(filtered_sentences)

In [242]:
#wordfreq for game 1
wordfreq = {}
for sentence in filtered_sentences[0]:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print(wordfreq)            

{'Quinns': 7, 'Let': 2, 'talk': 1, 'briefly': 1, 'Blackout': 4, 'Hong': 5, 'Kong': 5, 'eurogame': 2, 'Alexander': 2, 'Pfister': 3, 'I': 34, 'got': 7, 'excited': 5, 'couple': 1, 'episodes': 1, 'ago': 1, 'game': 10, 'running': 2, 'organization': 1, 'power': 2, 'cut': 1, 'keep': 3, 'city': 1, 'medicine': 3, 'food': 6, 'flowing': 2, 'You': 2, 'ta': 1, 'Matthew': 2, 'Matt': 6, 'And': 7, 'reasons': 1, 'every': 3, 'right': 3, 'actually': 2, 'coolest': 1, 'mechanics': 3, 'evocative': 1, 'cool': 4, 'The': 2, 'whole': 1, 'thing': 5, 'send': 1, 'people': 4, 'scout': 1, 'recon': 1, 'time': 1, 'lose': 1, 'somebody': 1, 'group/gang': 1, 'depending': 1, 'kind': 3, 'unsavory': 1, 'types': 2, 'hiring': 1, 'Very': 1, 'interesting': 4, 'We': 3, 'played': 8, 'bunch': 1, 'Philadelphia': 1, 'honest': 1, 'found': 1, 'It': 4, 'competent': 1, 'puzzle': 2, 'Mr': 1, 'thematic': 5, 'flair': 1, 'Great': 5, 'Western': 5, 'Trail': 5, 'This': 2, 'really': 8, 'public': 1, 'service': 1, 'announcement': 1, 'think': 4, '

In [243]:
#printing top 50 most frequent words for boardgame 1
most_freq = heapq.nlargest(50, wordfreq, key=wordfreq.get)
print(most_freq)

['I', 'like', 'game', 'different', 'played', 'really', 'Quinns', 'got', 'And', 'get', 'food', 'Matt', 'gon', 'na', 'make', 'Hong', 'Kong', 'excited', 'thing', 'thematic', 'Great', 'Western', 'Trail', 'A', 'But', 'Blackout', 'cool', 'people', 'interesting', 'It', 'think', 'games', '–', 'know', 'go', 'resources', 'Pfister', 'keep', 'medicine', 'every', 'right', 'mechanics', 'kind', 'We', 'pretty', 'yeah', 'Feast', 'For', 'Odin', 'would']


## Webscrapping

In [7]:
headers = {'User-Agent': 'Text mining'}

# Construct the results table
boardgameList = ['Just One']
listhashed = [i.replace(' ', '%20') for i in boardgameList]
summaryTable = pd.DataFrame()

for j in listhashed:
    # Load the webpage
    
    r = requests.get('https://boardgamegeek.com/geeksearch.php?action=search&objecttype=boardgame&q={}'.format(j), headers=headers)
    
    # Convert to BS
    bs = BeautifulSoup(r.text)
    
    # Find the right table
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('cellpadding'))
    #mydivs = bs.findAll("div", {"class": "table-responsive"})
    errors = bs.findAll(text="No Items Found")
   
    if not errors:
           
        tableHTML = table[0]
        #tableHTML = mydivs[0]
        # Convert the HTML of the table to pandas (very usefull function)
        pandasTable = pd.read_html(str(tableHTML), header=0)[0]

        # Extract all the links
        urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]
        urls = pd.DataFrame(urls)
        urls = urls[0][6]

        # Merge the links into the DF
        pandasTable['link'] = urls

        # Drop unused columns
        pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)

        # Create or extend the existing summary table
        if summaryTable.empty:
            summaryTable = pandasTable
        else:
            summaryTable = pd.concat([summaryTable, pandasTable],ignore_index=True)
    else:
        next

In [8]:
display(summaryTable)

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link
0,219.0,Just One (2018),7.209,7.61,6196,/boardgame/254640/just-one
1,NaN,Port Royal: Just One More Contract... (2015),6.549,7.53,1461,/boardgame/254640/just-one
2,NaN,Just One Die Soccer (2009),NaN,1.00,1,/boardgame/254640/just-one
3,NaN,Just One: Deutscher Spielepreis Classic Goodie...,NaN,6.64,7,/boardgame/254640/just-one
4,NaN,Just One: Gathering of Friends Promo Cards (2019),NaN,7.00,3,/boardgame/254640/just-one
5,NaN,Cocks Abreast Hostility: Cock Pack One – Just ...,NaN,7.71,7,/boardgame/254640/just-one


In [9]:
# Loading the firefox webdriver
driver = webdriver.Chrome('/Users/korneliusz.krysiak/Dropbox/TextMining/chromedriver')

In [10]:
# Construct the results table
detailedTable = pd.DataFrame(columns=['Title','Average Rating','Number of ratings','Std.dev','Number of Comments','Fans','Page Views','Overall Ranking','All Time Plays','Played This Month', 'Own', 'Previously Owned','For Trade','Want in Trade','Wishlist','Has Parts','Want Parts'])
detailedTable2 = pd.DataFrame()

for j in range(1):
    # Load the webpage
    driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][j]))
    
    # Get the source code and make it BS
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource, "lxml")
    
    # Find the relevant table
    data = bs.findAll('div',{'class':'outline-item-description'})
    
    # Loop over and strip spaces
    data2 = []
    for i in data:
        data2.append(i.get_text().strip())

    # Remove the ',' 
    for i in range(len(data2)):
        data2[i] = data2[i].replace(",","")

    # Wrangle the columns
    data2[3] = data2[3][0:4]
    data2[0] = float(data2[0])
    data2[1] = int(data2[1]) 
    data2[2] = float(data2[2])
    data2[3] = float(data2[3]) 

    # Convert to int
    for i in range(len(data2[4:])+1):
        i = i+3
        data2[i] = int(data2[i])

    # Special case - two categories. We want to drop both
    if len(data2) == 19:
        data2.pop(8)
        data2.pop(8)
    else:
        data2.pop(8)
    
    # Insert the data
    data2.insert(0,summaryTable['Title'][j]) 
    detailedTable.loc[len(detailedTable)] = data2

ValueError: invalid literal for int() with base 10: '53  Find For-Trade Matches'

In [11]:
display(detailedTable)

,Title,Average Rating,Number of ratings,Std.dev,Number of Comments,Fans,Page Views,Overall Ranking,All Time Plays,Played This Month,Own,Previously Owned,For Trade,Want in Trade,Wishlist,Has Parts,Want Parts


In [ ]:
# twoja kolej Michal